# Stanford SQL Course

This are personal notes about the SQL course offered by Stanford Lagunita.

The url of the course is https://lagunita.stanford.edu/courses/DB/SQL/SelfPaced/course/

In [2]:
# Showing content of repo
!ls

colleges.db  readme.md	stanford_sql_notes.Rmd	stanford_sql_notes.ipynb


In [3]:
# Importing neccessary libraries
import sqlite3
import pandas as pd

In [6]:
# Writting helper functions

db = 'colleges.db'

def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    return run_query('''
    select *
    from sqlite_master
    where type in ('table', 'view')
    ''')


In [8]:
# Showing tables
show_tables()

,type,name,tbl_name,rootpage,sql
0,table,College,College,2,"CREATE TABLE College(cName text, state text, e..."
1,table,Student,Student,3,"CREATE TABLE Student(sID int, sName text, GPA ..."
2,table,Apply,Apply,4,"CREATE TABLE Apply(sID int, cName text, major ..."


In [5]:
# Displaying necessary tables
pd.read_sql_query('''
select * from sqlite_master where type in ('table', 'view')
''', conn)

,type,name,tbl_name,rootpage,sql
0,table,College,College,2,"CREATE TABLE College(cName text, state text, e..."
1,table,Student,Student,3,"CREATE TABLE Student(sID int, sName text, GPA ..."
2,table,Apply,Apply,4,"CREATE TABLE Apply(sID int, cName text, major ..."


## Basic Select Statements

In [10]:
# Example of the basic select statement

run_query('''
    select sID, sName, GPA
    from Student
    where GPA > 3.6
''')

,sID,sName,GPA
0,123,Amy,3.9
1,456,Doris,3.9
2,678,Fay,3.8
3,987,Helen,3.7
4,876,Irene,3.9
5,654,Amy,3.9


In [13]:
# Using a select statement to perform an inner join

run_query('''
    select sName, major
    from Student, Apply
    where Student.sID = Apply.sID
''')

,sName,major
0,Amy,CS
1,Amy,CS
2,Amy,EE
3,Amy,EE
4,Bob,biology
5,Craig,CS
6,Craig,EE
7,Craig,bioengineering
8,Craig,bioengineering
9,Fay,history


In [14]:
# Using the 'distinct' keyword to get only unique results in the obtained relation

run_query('''
    select distinct sName, major
    from Student, Apply
    where Student.sID = Apply.sID
''')

,sName,major
0,Amy,CS
1,Amy,EE
2,Bob,biology
3,Craig,CS
4,Craig,EE
5,Craig,bioengineering
6,Fay,history
7,Helen,CS
8,Irene,CS
9,Irene,biology


In [15]:
# Use of the "and" operator to specify multiple conditions on a 'where' clause

run_query('''
    select sName, GPA, decision
    from Student, Apply
    where Student.sID = Apply.sID
        and sizeHS < 1000
        and major = 'CS'
        and cname = 'Stanford';
''')

,sName,GPA,decision
0,Helen,3.7,Y
1,Irene,3.9,N


In [20]:
# When performing joins is important to check any ambiguous column names

run_query('''
    select College.cName 
    from College, Apply
    where College.cName = Apply.cName
        and enrollment > 20000
        and major = 'CS'
''')

,cName
0,Berkeley
1,Cornell
2,Berkeley


In [21]:
# Joining three relations were table A is related to table B, and table B is related to table C

run_query('''
    select sName, GPA, Apply.cName
    from Student, College, Apply
    where Student.sID = Apply.sID and Apply.cName = College.cName;
''')

,sName,GPA,cName
0,Amy,3.9,Berkeley
1,Amy,3.9,Cornell
2,Amy,3.9,Stanford
3,Amy,3.9,Stanford
4,Bob,3.6,Berkeley
5,Craig,3.5,Cornell
6,Craig,3.5,Cornell
7,Craig,3.5,Cornell
8,Craig,3.5,MIT
9,Fay,3.8,Stanford


In [23]:
# Order the results of a query using the 'order by' clause and the 'desc'keyword

# When ordering by two different attributes, the first given will define the first sort
# and the second attribute will be applied afterwards
run_query('''
    select sName, GPA, Apply.cName
    from Student, College, Apply
    where Student.sID = Apply.sID and Apply.cName = College.cName
    order by sName desc, GPA
''')

,sName,GPA,cName
0,Jay,2.9,Cornell
1,Jay,2.9,Cornell
2,Jay,2.9,Stanford
3,Irene,3.9,MIT
4,Irene,3.9,MIT
5,Irene,3.9,Stanford
6,Helen,3.7,Berkeley
7,Helen,3.7,Stanford
8,Fay,3.8,Stanford
9,Craig,3.4,MIT


In [24]:
# The 'like' operator can be used inside a 'where' clause to select 

run_query('''
    select major, sID
    from Apply
    where major like '%eng%'
''')


,major,sID
0,bioengineering,345
1,bioengineering,345


In [25]:
# On the select clause it's also possible to perform arithmetic operations

run_query('''
    select gpa * 100
    from Student;

''')

,gpa * 100
0,390.0
1,360.0
2,350.0
3,390.0
4,290.0
5,380.0
6,340.0
7,370.0
8,390.0
9,290.0


In [27]:
# You can use aliases to rename any attribute on the resulting relation

run_query('''
    select gpa * 100 as upgraded_gpa
    from Student;

''')

,upgraded_gpa
0,390.0
1,360.0
2,350.0
3,390.0
4,290.0
5,380.0
6,340.0
7,370.0
8,390.0
9,290.0


In [ ]:
## 